In [1]:
import numpy as np
import pandas as pd

In [68]:
class Investment:

    def __init__(self, invested, round):
        self.invested = invested
        self.round = round
        self.shares = self.invested / self.round.price
        self.share_type = self.round.share_type

class Investor:

    def __init__(self, name):
        self.name = name
        self.investments = []

    def new_investment(self, invested, round):
        investment = Investment(invested, round)
        self.investments.append(investment)

    def total_shares(self):
        shares = 0
        for i in self.investments: 
            shares += i.shares 
        return shares

    def shares_ownership(self):
        stake = {}
        for i in self.investments:
            if i.share_type in stake.keys():
                stake[i.share_type] += i.shares
            else:
                stake[i.share_type] = i.shares
        return stake
    
    def total_invested(self):
        invested = 0
        for i in self.investments:
            invested += i.invested 
        return invested

class Covenants:
    
    def __init__(self, covenants={}):
        self.liquidity_preference = covenants.get('liquidity_preference', 0)
           
class Round:

    def __init__(self, name, invested, valuation, shares, type, covenants):
        self.name = name
        self.invested = invested
        self.valuation = valuation
        self.shares = shares
        self.share_type = type
        self.price = valuation/shares
        self.covenants = covenants

class BookInvestor:
    def __init__(self, name, stake):
        self.name = name
        self.stake = stake / 100

class Deal:

    def __init__(self, investment, stake, share_type):
        self.investment = investment
        self.stake = stake / 100
        self.share_type = share_type
        self.book = []

    def add_investor(self, investor):
        self.book.append(investor)

class Company:
    
    def __init__(self, name, equity, shares):
        self.name = name

        self.incorporation = Round('incorporation', equity, equity, shares, 'common', Covenants())

        self.founder = Investor('founder')
        self.founder.new_investment(equity, self.incorporation)

        self.investors = [self.founder]
        self.rounds = [self.incorporation]

    def current_stake(self):
        cap = []
        for i in self.investors:
            last_round = self.rounds[-1]
            total_shares = last_round.shares
            investor_shares = i.total_shares()
            cap.append({
                'investor': i.name,
                'stake': investor_shares / total_shares * 100
            })
        return cap
    
    def shares_ownership(self):
        cap = []
        total_shares = self.rounds[-1].shares
        for i in self.investors:
            investor_ownership = i.shares_ownership()
            for j in investor_ownership.keys():
                investor = {}
                investor['name'] = i.name
                investor['shares_type'] = j
                investor['shares'] = investor_ownership[j]
                investor['stake'] = investor['shares'] / total_shares
                cap.append(investor)
        return cap

    def get_investor(self, name):
        for i in self.investors:
            if name == i.name:
                return i
        return None

    def add_investor(self, name):
        new_investor = Investor(name)
        self.investors.append(new_investor)
        return new_investor

    def get_add_investor(self, name):
        investor = self.get_investor(name)
        if investor is None:
            investor = self.add_investor(name)
        return investor

    def new_round(self, name, deal, covenants):

        post_money_valuation = deal.investment / deal.stake
        pre_money_valuation = post_money_valuation -  deal.investment
        
        total_shares =  self.rounds[-1].shares * ( 1 + deal.investment / pre_money_valuation)
        
        round = Round(name, deal.investment, post_money_valuation, total_shares, deal.share_type, covenants = covenants)
        
        self.rounds.append(round)

        for i in deal.book:
            new_investor = self.get_add_investor(i.name)
            new_investor.new_investment(deal.investment * i.stake , round)
            
        return None

In [69]:
capital = 1e4
price_per_share = 0.01

company = Company('consorciei', capital, capital/price_per_share)
company.incorporation.valuation

10000.0

In [70]:
cap = company.current_stake()
cap

[{'investor': 'founder', 'stake': 100.0}]

In [71]:
deal = Deal(10000, 10, 'common')
deal.add_investor(BookInvestor('ze', 100))
company.new_round('seed', deal, Covenants({}))

deal = Deal(500000, 20, 'a')
deal.add_investor(BookInvestor('tera', 80))
deal.add_investor(BookInvestor('patria', 20))
covenants = Covenants({'liquidity_preference': 2})
company.new_round('a', deal, covenants)

In [72]:
company.current_stake()

[{'investor': 'founder', 'stake': 72.0},
 {'investor': 'ze', 'stake': 8.000000000000002},
 {'investor': 'tera', 'stake': 16.0},
 {'investor': 'patria', 'stake': 4.0}]

In [73]:
company.investors[3].shares_ownership()

{'a': 55555.55555555556}

In [74]:
company.shares_ownership()

[{'name': 'founder',
  'shares_type': 'common',
  'shares': 1000000.0,
  'stake': 0.72},
 {'name': 'ze',
  'shares_type': 'common',
  'shares': 111111.11111111114,
  'stake': 0.08000000000000002},
 {'name': 'tera',
  'shares_type': 'a',
  'shares': 222222.22222222225,
  'stake': 0.16},
 {'name': 'patria',
  'shares_type': 'a',
  'shares': 55555.55555555556,
  'stake': 0.04}]

In [ ]:
exit_multiple = 1
last_valuation = company.rounds[-1].valuation 
total_shares = company.rounds[-1].shares
exit_valuation = last_valuation*exit_multiple



In [75]:
exit_multiple = 1
last_valuation = company.rounds[-1].valuation 
total_shares = company.rounds[-1].shares
exit_valuation = last_valuation*exit_multiple

val_to_dist = exit_valuation
aux_investments = []
for i in company.investors:
    for a in i.investments:
        aux_investment = {
            'name': i.name,
            'round': a.round.name,
            'invested': a.invested,
            'shares': a.shares,
            'stake': a.shares/total_shares,
            'round_size': a.round.invested,
            'round_share': a.invested/a.round.invested,
            'liq_pref': a.round.covenants.liquidity_preference,
            'stake_value': a.shares/total_shares*exit_valuation,
            'liq_pref_floor':  a.invested * a.round.covenants.liquidity_preference
        }
        aux_investments.append(aux_investment)

total_no_preference_shares = 0 

for r in company.rounds[::-1]:
    r_invested = r.invested
    r_liq_pref = r.covenants.liquidity_preference
    r_to_pay = np.minimum(r_invested * r_liq_pref, val_to_dist)
    for k,i in enumerate(aux_investments):
        if i['round'] == r.name and r.covenants.liquidity_preference:
            stake_value = i['stake_value']
            liquidity_pref_floor = np.minimum(i['invested']*i['liq_pref'], r_to_pay * i['round_share'])
            total_paid = np.minimum(np.maximum(liquidity_pref_floor, stake_value), val_to_dist)
            aux_investments[k]['total_paid']  = total_paid
            aux_investments[k]['paid_due_to_stake'] = stake_value
            aux_investments[k]['paid_due_to_pref'] = aux_investments[k]['total_paid']-stake_value
            val_to_dist -= total_paid
        else:
            total_no_preference_shares += i['shares']

for r in company.rounds[::-1]:          
    for k,i in enumerate(aux_investments):
        if i['round'] == r.name and not r.covenants.liquidity_preference:
            aux_investments[k]['paid_due_to_stake'] = i['stake_value']
            aux_investments[k]['total_paid'] =  i['shares'] / total_no_preference_shares * val_to_dist
            aux_investments[k]['paid_due_to_pref'] = aux_investments[k]['total_paid']-i['stake_value']
            
print('---------------------------------------------')
print('last valuation: %s' % last_valuation)
print('exit valuation: %s' % exit_valuation)
print('---------------------------------------------')
print('name', '|', 'round', '|', 'invested', '|', 'stake', '|', 'total_paid')
print('stake_value', '|', 'liq_pref_floor', '|', 'paid_due_to_pref', '|', 'paid_due_to_stake')
print('------------------------------------------------------')
for i in aux_investments:
    print(i['name'], '|', i['round'], '|', i['invested'], '|', i['stake'], '|', i['total_paid'])
    print(i['stake_value'], '|', i['liq_pref_floor'], '|', i['paid_due_to_pref'], '|', i['paid_due_to_stake'])
    print('------------------------------------------------------')

---------------------------------------------
last valuation: 2500000.0
exit valuation: 2500000.0
---------------------------------------------
name | round | invested | stake | total_paid
stake_value | liq_pref_floor | paid_due_to_pref | paid_due_to_stake
------------------------------------------------------
founder | incorporation | 10000.0 | 0.72 | 385714.28571428574
1800000.0 | 0.0 | -1414285.7142857143 | 1800000.0
------------------------------------------------------
ze | seed | 10000.0 | 0.08000000000000002 | 42857.14285714287
200000.00000000003 | 0.0 | -157142.85714285716 | 200000.00000000003
------------------------------------------------------
tera | a | 400000.0 | 0.16 | 800000.0
400000.0 | 800000.0 | 400000.0 | 400000.0
------------------------------------------------------
patria | a | 100000.0 | 0.04 | 200000.0
100000.0 | 200000.0 | 100000.0 | 100000.0
------------------------------------------------------
